In [1]:
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:99% !important; }</style>"))

import multiprocessing
import azimuth.model_comparison
import time

import sys
print(sys.version)

2.7.16 |Anaconda, Inc.| (default, Mar 14 2019, 21:00:58) 
[GCC 7.3.0]


/home/tdfair/anaconda3/envs/py27/lib/python2.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


# Load 500 random 30mers with PAM site

In [2]:
random_30 = pd.read_csv('/home/tdfair/Documents/Azimuth_random_sgRNAs_updated_sets/random_sgRNAs_Azimuth/around80_random', sep='\t').sample(n=500).reset_index(drop=True)
random_30.head()

,sgRNAs
0,GTAGACTCTCCTTAGAATCCCCCGCGGAGT
1,TGGAATGGTCTAAGATCTAAACCGCGGCAC
2,GTGCAGGAAAGCAAGAGTTCACCGCGGTTG
3,GTTGGCTAGCGTTAGAAAACACCGCGGTAT
4,AACCTGTCTTCTGAGACAACAAGGCGGTGC


# Apply Rule Set 2 (Azimuth) scoring with multiprocessing support 

Doench et al., *Nature Biotechnology* 2016

In [3]:
def Azimuth(seq):
    amino_acid_cut_positions = None
    percent_peptides = None
    scores = azimuth.model_comparison.predict(seq, amino_acid_cut_positions, percent_peptides)
    return scores

def parallelize_dataframe(df, func):
    print 'CPU count:', multiprocessing.cpu_count()-1
    df_split = np.array_split(df, multiprocessing.cpu_count()-1)
    pool = multiprocessing.Pool(multiprocessing.cpu_count()-1)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

def parallelize_Azimuth(seq):
    scores = Azimuth(seq)
    scores = pd.DataFrame(scores, columns=['Azimuth_parallel'])
    return scores

#Convert the 'sgRNAs' column to a Numpy array:

sgRNA30 = random_30['sgRNAs'].to_numpy()

#Run parallelized Azimuth scoring:

start = time.time()
Azimuth = parallelize_dataframe(sgRNA30, parallelize_Azimuth)
end = time.time()
print 'Elapsed time:', end - start 

#Add scores back to DataFame:

random_30['Azimuth'] = Azimuth['Azimuth_parallel'].tolist()

CPU count: 23
No model file specified, using V3_model_nopos
No model file specified, using V3_model_nopos
No model file specified, using V3_model_nopos
No model file specified, using V3_model_nopos
No model file specified, using V3_model_nopos
No model file specified, using V3_model_nopos
No model file specified, using V3_model_nopos
No model file specified, using V3_model_nopos
No model file specified, using V3_model_nopos
No model file specified, using V3_model_nopos
No model file specified, using V3_model_nopos
No model file specified, using V3_model_nopos
No model file specified, using V3_model_nopos
No model file specified, using V3_model_nopos
No model file specified, using V3_model_nopos
No model file specified, using V3_model_nopos
No model file specified, using V3_model_nopos
No model file specified, using V3_model_nopos
No model file specified, using V3_model_nopos
No model file specified, using V3_model_nopos
No model file specified, using V3_model_nopos
No model file specif

/home/tdfair/anaconda3/envs/py27/lib/python2.7/site-packages/azimuth/features/featurization.py:144: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  feat_NX = pandas.concat([feat_NX, NX_onehot], axis=1)
/home/tdfair/anaconda3/envs/py27/lib/python2.7/site-packages/azimuth/features/featurization.py:144: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  feat_NX = pandas.concat([feat_NX, NX_onehot], axis=1)
/home/tdfair/anaconda3/envs/py27/lib/python2.7/site-packages/azimuth/features/featurization.py:144: FutureWarning: Sorting because non-concatenation axis is not aligned

/home/tdfair/anaconda3/envs/py27/lib/python2.7/site-packages/azimuth/features/featurization.py:144: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  feat_NX = pandas.concat([feat_NX, NX_onehot], axis=1)
/home/tdfair/anaconda3/envs/py27/lib/python2.7/site-packages/azimuth/features/featurization.py:144: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  feat_NX = pandas.concat([feat_NX, NX_onehot], axis=1)
/home/tdfair/anaconda3/envs/py27/lib/python2.7/site-packages/azimuth/features/featurization.py:144: FutureWarning: Sorting because non-concatenation axis is not aligned

Elapsed time: 0.466323852539


In [4]:
random_30.sort_values('Azimuth', ascending=False).head()

,sgRNAs,Azimuth
42,CGGCGTAATGCCGAGACTACACAGCGGTCT,0.963440
11,TGCAGTACGTATGAGAGACCACAGCGGTCA,0.921669
114,AAGGTATATGTCGAGATAACACAGCGGTCC,0.918181
48,AGGTTGACTGCGAAGAATGCACTGCGGTCG,0.914894
490,TCCGGAGGTGGAGAGATTACAACGCGGTCT,0.911319
